# Amphi 8 - More about NLP [2]: Applications

# 1. Part of Speech Tagging

# 2. Name Entity Recognition

# 3. Machine Translation

## 3.1 Sequence-To-Sequence Model

Remind that sequence-to-sequence models are RNNs that follow the below schema

<img src="F1.png"></img>

In terms of probability, we would like to modelize the probability

$$
\mathbf P(y^{<1>}, \ldots, y^{<T_y>} |x^{<1>}, \ldots, x^{<T_x>})
$$

These models are used in case of machine translation.

## 3.2 The Problem

Suppose we have a French sentence that can be translated to English as follows:
<table>
    <tr>
        <td>
            $x^{<1>}$
        </td>
        <td>
            $x^{<2>}$
        </td>
        <td>
            $x^{<3>}$
        </td>
        <td>
            $x^{<4>}$
        </td>
        <td>
            $x^{<5>}$
        </td>
        <td></td>
    </tr>
    <tr>
        <td>
            Jane
        </td>
        <td>
            visite
        </td>
        <td>
            l'Afrique
        </td>
        <td>
            en
        </td>
        <td>
            Septembre
        </td>
        <td></td>
    </tr>
    <tr>
        <td>
            $y^{<1>}$
        </td>
        <td>
            $y^{<2>}$
        </td>
        <td>
            $y^{<3>}$
        </td>
        <td>
            $y^{<4>}$
        </td>
        <td>
            $y^{<5>}$
        </td>
        <td>
            $y^{<6>}$
        </td>
    </tr>
    <tr>
        <td>
            Jane
        </td>
        <td>
            is
        </td>
        <td>
            visiting
        </td>
        <td>
            Africa
        </td>
        <td>
            in
        </td>   
        <td>
            September
        </td>
    </tr>
</table>

As seen in section 3.1, sequence-to-sequence models can be used for this translation problem. We can feed the French sentence as input and the English one as output. If the training set (corpus) is large enough, practice experiences show that the model works well for the translation of new sentences.

The big problem of this model is that the dimensionality of output is exponentially large. If we have a vocabulary of size $V$ then we ends up with $|V|^{T_y^{max}}$ possibilities of output when looking for all possibility of sentences of no more than $T_y^{max}$ words. We can reduce this complexity by an **approximate search out** algorithm.

## 3.3 Beam Search

### 3.3.1 Basic Algorithm

**Beam Search** is an example of **approximate search out** algorithm.

Supposing we have a vocabulary of $|V|=10000$ words.

<p align="center">
<table>
    <tr>
        <td>
            a
        </td>
    </tr>
    <tr>
        <td>
            ...
        </td>
    </tr>
    <tr>
        <td>
            in
        </td>
    </tr>
    <tr>
        <td>
            ...
        </td>
    </tr>
    <tr>
        <td>
            jane
        </td>
    </tr>
    <tr>
        <td>
            ...
        </td>
    </tr>
    <tr>
        <td>
            september
        </td>
    </tr>
    <tr>
        <td>
            ...
        </td>
    </tr>
    <tr>
        <td>
            zyzy
        </td>
    </tr>
</table>
    
We can calculate
$$
\mathbf P(y^{<1>}|x)
$$

for all words $y^{<1>}$ in the vocabulary.

**Step 1**

Beam search suggests picking up **3 words** with the highest conditional probabilities. For example, after calculating probability for the first word, it finds that the 3 best suitable words are "in", "jane" and "september"


**Step 2**

With each of these 3 choices of $y^{<1>}$, say $y^{<1>}_{option 1}, \ldots, y^{<1>}_{option 3}$, it considers the second word and calculate
$$
\mathbf P(y^{<2>}| x, y^{<1>}_{option i})
$$
Now, we can deduce
$$
\mathbf P(y^{<1>}, y^{<2>}|x) = \mathbf P(y^{<1>}|x) \mathbf P(y^{<2>}| x, y^{<1>})
$$

After calculating the $3|V|$ = 30000 possibilities, it chooses the best 3 pairs of $(y^{<1>}, y^{<2>})$. For example, "in september", "jane is" and "jane visits".

...

**Repeat the steps until the "END" token appears in all of the 3 possibilities.**

### 3.3.2 Refinement of Beam Search

**Length Normalization**

We talked about Beam Search as maximizing the probability

$$
\arg\max_y \prod_{t=1}^{T_y} \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

This probability is very small. Instead, we use the log form.

$$
\arg\max_y \sum_{t=1}^{T_y} \log \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

Although we work with logarithm, very long sentences can reduce the performance of the algorithm. Hence, although it seems not natural, we can avoid having long translated sentences by re-evaluating

$$
\sum_{t=1}^{T_y} \frac1{T_y} \log \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

or
$$
\sum_{t=1}^{T_y} \frac1{T_y^\alpha} \log \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

with some predefined $\alpha$.

**Modified Algorithm for Length Normalization**

- Do a standard Beam Search with 
$$
\sum_{t=1}^{T_y}\log \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

- Then, at the end, we retrieve for example 3 sentences of length 1, 3 sentences of length 2, ..., 3 sentences of length 29, 2 sentences of length 30, 2 sentences of length 31, 1 sentences of length 32. For these 92 sentences, calculate

$$
\sum_{t=1}^{T_y} \frac1{T_y^\alpha} \log \mathbf P(y^{<t>}|x, y^{<1>}, \ldots, y^{<t-1>})
$$

and choose the one that maximize that modified object function.

**The beam width**

In the algorithm we chose the beam width $B=3$. It is possible to choose other $B$. Larger $B$ gives better results, but make the algorithm slower, while small $B$ inversely.

People often try $B = 1, 3, 10, 100$.

Finally, keep in mind that this is just an approximate search out algorith. It does not guarantee to find the exact maximum for $\arg\max_y P(y|x)$

### 3.3.3 Error Analysis in Beam Search

Suppose we have a French sentence

`Jane visite l'Afrique en Septembre.`

We expect a translation like `Jane visits Africa in September.`

But somehow, the machine algorithm predicts: `Jane visited Africa last September.`

We would like to blame the error to one of the two elements:

- The RNN model
- The Beam Search algorithm

What should we blame on?

Let $y^\star$ denote the expected answer ( `Jane visits Africa in September.`) and $\hat y$ denote the machine's answer (`Jane visited Africa last September.`).

In fact, RNN computes $\mathbf P(y|x)$. We can compute $\mathbf P(y^\star|x)$, $\mathbf P(\hat y|x)$.

**Case 1**
$$\mathbf P(y^\star|x) > \mathbf P(\hat y|x)$$
and Beam search chose $\hat y$.

Conclusion: Beam search is at fault.

Solution: Try with larger $B$

**Case 2**
$$\mathbf P(y^\star|x) \leq \mathbf P(\hat y|x)$$
But RNN predicted $P(y^\star|x) < P(\hat y|x)$

Conclusion: RNN model is at fault.

Solution: Try training more data, or regularize.

## 3.4 Evaluate Machine Translation. BLEU Score

BLEU = Bilingual evaluation understudy

- French: Le chat est sur le tapis.
- Reference 1: The cat is on the mat.
- Reference 2: There is a cat on the mat.
- MT Output: the the the the the the the.

Calculate the precision for one word:
- Denominator: Count of a word in the MT output
- Numerator: Max of a word in the reference
- Precision: $\frac27$

Calculate the precision for a pair of words:

- Reference 1: The cat is on the mat.
- Reference 2: There is a cat on the mat.
- MT Output: the the the the the the the.

- Denominator: Count of a word in the MT output
- Numerator: Max of a word in the reference
- Precision: $\frac27$

|         | count | max count in reference | Precision |
|---------|-------|------------------------|-----------|
| the cat | 2     | 1                      | $\frac12$ |
| cat the | 1     | 0                      | 0         |
| cat on  | 1     | 1                      | 1         |
| on the  | 1     | 1                      | 1         |
| the mat | 1     | 1                      | 1         |

**BLEU score** on unigram

$$
p_1 =\frac{\sum_{unigram \in \hat y} count_{ref}(unigram)}{\sum_{unigram\in \hat y}count(unigram)}
$$

**BLEU score** on n-gram

$$
p_1 =\frac{\sum_{n-gram \in \hat y} count_{ref}(n-gram)}{\sum_{n-gram\in \hat y}count(n-gram)}
$$

**Combined bleu score**
$$
BP = \exp(\frac14 \sum_{n=1}^4 p_n)
$$

BLEU score is used to evaluate if a machine translation is good.

## 3.5 Attention Model

The problem of long sequences is it takes very long history to memorize and would not able to perform well a decoding on translation step.

Normally that's what human does: split sentences into parts, generate parts of the translation then combine them in sequence.

The **Attention Model** try to look at part of the sentence at a time.

<img src="F2.png"></img>

# 4. Speech Recognition Problem

In speech recognition problem, $x$ is an audioclip and $y$ is a transcipt.

A preprocessing step can translate the raw clip to spectrogram (false back output). The human ear does the computation quite likely the same as this preprocessing.

Nowadays, the preprocessing is not necessary. Instead, people use large dataset (10000h of audio) for training and the model (sequence-to-sequence, attention model) will be able to directly generate the transcript.

## 4.1 CTC Cost for Speech Recognition

If your audio has frequency 100 Hrtz in 10s, we will have 1000 inputs.
It will not output "the quick brown fox", but something like that
```
ttttttttttttt_h_eeeeeee_______[ ]_________qqq__...
```

## 4.2 Trigger Word Decision

Trigger word = "OK google", "Hey Siri"

## References

[1]

[2]

[3] Panneni et al., 2002, Bleu: A method for automatic evaluation on machine translation.

[4] Najdanau et al., 2014, Neural machine translation by jointly learning to align and translate

[5] https://medium.com/machine-learning-bites/deeplearning-series-attention-model-and-speech-recognition-deeb50632152